In [3]:
# Import necessary libraries
from ultralytics import YOLO
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from PIL import Image as PILImage
import io
from IPython.display import Image, display
from PIL import Image as PILImage
import numpy as np
import tensorflow as tf
import io
import os
import pandas as pd
import torch
import json
from torch import float32, tensor
import torchmetrics
import cv2


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from tqdm import tqdm

weights=models.VGG19_Weights.IMAGENET1K_V1
model = models.vgg19(weights=weights)
preprocess = weights.transforms()
model.classifier[6] = nn.Linear(4096, 29)
model.load_state_dict(torch.load('./models/vgg19_model.pth'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
# Load the YOLO model
yolo_model = YOLO('./YOLOv8_training/best.pt').to(device)

# Class mapping
class_mapping = {'Apple Scab Leaf': 0, 'Apple leaf': 1, 'Apple rust leaf': 2, 'Bell_pepper leaf': 3, 'Bell_pepper leaf spot': 4, 'Blueberry leaf': 5, 'Cherry leaf': 6, 'Corn Gray leaf spot': 7, 'Corn leaf blight': 8, 'Corn rust leaf': 9, 'Peach leaf': 10, 'Potato leaf': 11, 'Potato leaf early blight': 12, 'Potato leaf late blight': 13, 'Raspberry leaf': 14, 'Soyabean leaf': 15, 'Squash Powdery mildew leaf': 16, 'Strawberry leaf': 17, 'Tomato Early blight leaf': 18, 'Tomato Septoria leaf spot': 19, 'Tomato leaf': 20, 'Tomato leaf bacterial spot': 21, 'Tomato leaf late blight': 22, 'Tomato leaf mosaic virus': 23, 'Tomato leaf yellow virus': 24, 'Tomato mold leaf': 25, 'Tomato two spotted spider mites leaf': 26, 'grape leaf': 27, 'grape leaf black rot': 28}

# Directory containing images to predict
directory = './datasets/validation/images'
# directory = './TEST'

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
preds=[]
# Define a list of dictionaries
data = []
# Create empty lists to store results
resized_cropped_images = []
predicted_classes = []

In [15]:
df = pd.read_csv('./YOLOv8_training/new_labels.csv')
# df = pd.read_csv('./set.csv')
model.eval()
# Define the target size for ResNet50
target_size = (224, 224)
# Process YOLO results
for filename in os.listdir(directory):
    filtered_df = df[df['filename'] == filename]
    results = yolo_model.predict(os.path.join(directory,filename))
    for x in results:
        for r in x:
            # Get the image and bounding box
            im_array = r.plot()  # plot a BGR numpy array of predictions
            im = PILImage.fromarray(im_array[..., ::-1])  # Convert BGR to RGB PIL image
            bbox = r.boxes.data[0, :4].cpu().numpy()  # Get the bounding box

            # Crop the detected object from the image
            cropped_im = im.crop((bbox[0], bbox[1], bbox[2], bbox[3]))

            # Resize the cropped image to the target size
            resized_cropped_im = cropped_im.resize(target_size)
            # plt.imshow(resized_cropped_im)

            # Save the resized and cropped image to a BytesIO object
            img_io = io.BytesIO()
            resized_cropped_im.save(img_io, 'PNG')

            # Append the resized and cropped image to the list
            resized_cropped_images.append(resized_cropped_im)
            input_image = preprocess(cropped_im).unsqueeze(0).to(device)
            with torch.no_grad():
                output = model(input_image)
            logits = torch.nn.functional.softmax(output[0], dim=0)
            # model predicts one of the 1000 ImageNet classes
            predicted_class_index = torch.argmax(logits).item()
            predicted_class_label= list(class_mapping.keys())[predicted_class_index]
            predicted_classes.append(predicted_class_label)
            # Convert bounding box to tensor and move to GPU
            bbox_tensor = torch.tensor([[bbox[0], bbox[1], bbox[2], bbox[3]]]).to(device)
            
            # Convert labels to tensor and move to GPU
            label_tensor = torch.tensor([predicted_class_index]).to(device)

            # Convert scores to tensor and move to GPU
            score_tensor = torch.tensor([logits[predicted_class_index].item()]).to(device)*r.boxes.conf
#             score_tensor=r.boxes.conf
            minind=0
            q=0
            err=1e9
            for(index, row) in filtered_df.iterrows():
                # Access fields using the column names
                temerr=(bbox[0]-row['xmin'])**2+(bbox[1]-row['ymin'])**2+(bbox[2]-row['xmax'])**2+(bbox[3]-row['ymax'])**2
                if(temerr<err):
                    err=temerr
                    minind=q
                q+=1
            print(len(filtered_df), " ", minind)
            if(len(filtered_df)>0):
                data.append({'boxes': torch.tensor([[(filtered_df.iloc[minind]['xmin'].copy()), (filtered_df.iloc[minind]['ymin'].copy()), (filtered_df.iloc[minind]['xmax'].copy()), (filtered_df.iloc[minind]['ymax'].copy())]]).to(device), 'labels': torch.tensor([(class_mapping[filtered_df.iloc[minind]['class']])]).to(device)})
                # filtered_df.drop(filtered_df.index[minind], inplace=True)
                preds.append(dict(boxes=bbox_tensor, labels=label_tensor, scores=score_tensor))
    filtered_df=[]
    
    


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img2288.jpg: 480x640 3 leafs, 9.2ms
Speed: 1.0ms preprocess, 9.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
1   0
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img1074.jpg: 448x640 1 leaf, 7.5ms
Speed: 0.9ms preprocess, 7.5ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img2329.jpg: 288x640 2 leafs, 5.8ms
Speed: 0.6ms preprocess, 5.8ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 640)
2   0
2   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img928.jpg: 512x640 2 leafs, 8.9ms
Speed: 1.0ms preprocess, 8.9ms inference, 0.7ms postprocess per image at shape (1, 3, 512, 640)
1   0
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-d

1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img1453.jpg: 480x640 1 leaf, 9.4ms
Speed: 1.2ms preprocess, 9.4ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img1313.jpg: 480x640 3 leafs, 9.3ms
Speed: 1.0ms preprocess, 9.3ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
2   0
2   1
2   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img1273.jpg: 480x640 8 leafs, 9.1ms
Speed: 0.9ms preprocess, 9.1ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
4   0
4   2
4   3
4   3
4   3
4   1
4   3
4   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img1397.jpg: 480x640 2 leafs, 9.2ms
Speed: 1.0ms preprocess, 9.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
2   0
2   0

image 

4   0
4   2
4   3
4   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img1646.jpg: 416x640 2 leafs, 7.7ms
Speed: 0.7ms preprocess, 7.7ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 640)
9   0
9   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img2558.jpg: 640x640 11 leafs, 11.4ms
Speed: 1.4ms preprocess, 11.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
12   7
12   6
12   5
12   6
12   8
12   0
12   9
12   8
12   3
12   11
12   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img678.jpg: 480x640 1 leaf, 9.6ms
Speed: 1.1ms preprocess, 9.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img996.jpg: 480x640 1 leaf, 9.3ms
Speed: 0.6ms preprocess, 9.3ms inference, 0.5ms postprocess per image at 

5   4
5   2
5   0
5   3
5   1

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img193.jpg: 512x640 1 leaf, 8.5ms
Speed: 0.8ms preprocess, 8.5ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)
1   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img628.jpg: 480x640 5 leafs, 9.3ms
Speed: 0.9ms preprocess, 9.3ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
2   1
2   0
2   0
2   1
2   0

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets/validation/images/img2006.jpg: 640x480 26 leafs, 9.4ms
Speed: 1.4ms preprocess, 9.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)
21   0
21   0
21   2
21   0
21   1
21   4
21   1
21   3
21   10
21   5
21   17
21   13
21   20
21   6
21   11
21   4
21   15
21   18
21   8
21   1
21   4
21   12
21   8
21   17
21   0
21   7

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/

In [16]:
# for img_data, predicted_class in zip(resized_cropped_images, predicted_classes):
#     display(img_data, predicted_class, class_mapping[predicted_class])


In [17]:
metrics = torchmetrics.detection.MeanAveragePrecision().to(device)
for i in range(len(preds)):
    print([preds[i]], [data[i]])
    metrics.update([preds[i]], [data[i]])
print(metrics.compute())

[{'boxes': tensor([[305.9838, 293.8942, 437.3342, 479.8495]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.5633], device='cuda:0')}] [{'boxes': tensor([[312, 288, 445, 475]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[ 14.0796, 155.6635, 212.2745, 286.5225]], device='cuda:0'), 'labels': tensor([22], device='cuda:0'), 'scores': tensor([0.2844], device='cuda:0')}] [{'boxes': tensor([[ 25, 160, 204, 279]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[209.6182, 224.8037, 342.9072, 447.2449]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.3396], device='cuda:0')}] [{'boxes': tensor([[215, 240, 323, 430]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[238.8710,  19.7182, 340.7936, 225.7074]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.4006], device='cuda:0')}] [{'boxes': tensor([[249,  

[{'boxes': tensor([[577.8527, 290.0592, 797.3862, 402.6263]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'scores': tensor([0.3031], device='cuda:0')}] [{'boxes': tensor([[566, 289, 813, 394]], device='cuda:0'), 'labels': tensor([25], device='cuda:0')}]
[{'boxes': tensor([[  0.0000,   0.0000, 147.4210, 260.1210]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'scores': tensor([0.1601], device='cuda:0')}] [{'boxes': tensor([[ 76,   2, 401, 192]], device='cuda:0'), 'labels': tensor([25], device='cuda:0')}]
[{'boxes': tensor([[355.0282, 413.3159, 499.1038, 568.7329]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'scores': tensor([0.2360], device='cuda:0')}] [{'boxes': tensor([[346, 414, 685, 735]], device='cuda:0'), 'labels': tensor([25], device='cuda:0')}]
[{'boxes': tensor([[744.0559, 102.7337, 954.0316, 459.1070]], device='cuda:0'), 'labels': tensor([22], device='cuda:0'), 'scores': tensor([0.0612], device='cuda:0')}] [{'boxes': tensor([[ 641,   3

[{'boxes': tensor([[655.0626, 292.2093, 749.0308, 350.9451]], device='cuda:0'), 'labels': tensor([0], device='cuda:0'), 'scores': tensor([0.1721], device='cuda:0')}] [{'boxes': tensor([[686, 297, 835, 416]], device='cuda:0'), 'labels': tensor([25], device='cuda:0')}]
[{'boxes': tensor([[ 38.7731, 108.1730, 617.8079, 516.7414]], device='cuda:0'), 'labels': tensor([14], device='cuda:0'), 'scores': tensor([0.9360], device='cuda:0')}] [{'boxes': tensor([[ 45,  93, 611, 515]], device='cuda:0'), 'labels': tensor([14], device='cuda:0')}]
[{'boxes': tensor([[606.6337,  11.5561, 884.1423, 540.5334]], device='cuda:0'), 'labels': tensor([14], device='cuda:0'), 'scores': tensor([0.9236], device='cuda:0')}] [{'boxes': tensor([[595,  21, 880, 535]], device='cuda:0'), 'labels': tensor([14], device='cuda:0')}]
[{'boxes': tensor([[ 801.8220,  362.5487, 1284.5291,  742.0249]], device='cuda:0'), 'labels': tensor([14], device='cuda:0'), 'scores': tensor([0.9166], device='cuda:0')}] [{'boxes': tensor([[ 79

[{'boxes': tensor([[129.2222, 363.0611, 169.3110, 426.6584]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.8785], device='cuda:0')}] [{'boxes': tensor([[165, 135, 310, 212]], device='cuda:0'), 'labels': tensor([25], device='cuda:0')}]
[{'boxes': tensor([[209.9493, 373.6956, 262.3728, 430.9153]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.8758], device='cuda:0')}] [{'boxes': tensor([[165, 135, 310, 212]], device='cuda:0'), 'labels': tensor([25], device='cuda:0')}]
[{'boxes': tensor([[122.2677,  40.0707, 223.6062, 167.3556]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.8710], device='cuda:0')}] [{'boxes': tensor([[120,  37, 237, 162]], device='cuda:0'), 'labels': tensor([25], device='cuda:0')}]
[{'boxes': tensor([[591.6000,  58.2187, 719.3826, 171.9979]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.8447], device='cuda:0')}] [{'boxes': tensor([[478,  

[{'boxes': tensor([[7.6988e-01, 9.3671e+02, 2.1676e+02, 1.1349e+03]], device='cuda:0'), 'labels': tensor([20], device='cuda:0'), 'scores': tensor([0.2545], device='cuda:0')}] [{'boxes': tensor([[643, 571, 755, 824]], device='cuda:0'), 'labels': tensor([20], device='cuda:0')}]
[{'boxes': tensor([[1263.9458, 1011.7592, 1490.7635, 1199.6691]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.3338], device='cuda:0')}] [{'boxes': tensor([[1164,  703, 1417,  936]], device='cuda:0'), 'labels': tensor([20], device='cuda:0')}]
[{'boxes': tensor([[440.7644, 368.2744, 647.3232, 476.1044]], device='cuda:0'), 'labels': tensor([20], device='cuda:0'), 'scores': tensor([0.2826], device='cuda:0')}] [{'boxes': tensor([[449, 373, 638, 473]], device='cuda:0'), 'labels': tensor([20], device='cuda:0')}]
[{'boxes': tensor([[ 53.8589,  75.5080, 238.5710, 279.8344]], device='cuda:0'), 'labels': tensor([20], device='cuda:0'), 'scores': tensor([0.4898], device='cuda:0')}] [{'boxes':

...

[{'boxes': tensor([[ 869.6161,   94.0839, 1023.0000,  276.4231]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.8030], device='cuda:0')}] [{'boxes': tensor([[ 872,  110, 1023,  271]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[695.7000, 385.2042, 897.7627, 592.9650]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.7865], device='cuda:0')}] [{'boxes': tensor([[575, 295, 780, 488]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[8.0976e-03, 4.1176e+02, 1.4067e+02, 6.7182e+02]], device='cuda:0'), 'labels': tensor([17], device='cuda:0'), 'scores': tensor([0.4291], device='cuda:0')}] [{'boxes': tensor([[ 46, 117, 319, 336]], device='cuda:0'), 'labels': tensor([17], device='cuda:0')}]
[{'boxes': tensor([[ 915.8021,  360.0293, 1022.8818,  492.2874]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'scores': tensor([0.5553], device='cuda:0')}] [{'boxe

[{'boxes': tensor([[  0.0000,   2.7963, 135.5305, 250.7186]], device='cuda:0'), 'labels': tensor([28], device='cuda:0'), 'scores': tensor([0.8602], device='cuda:0')}] [{'boxes': tensor([[  1,   9, 136, 239]], device='cuda:0'), 'labels': tensor([28], device='cuda:0')}]
[{'boxes': tensor([[238.6522,  23.2224, 539.3741, 653.3717]], device='cuda:0'), 'labels': tensor([22], device='cuda:0'), 'scores': tensor([0.3752], device='cuda:0')}] [{'boxes': tensor([[249,  18, 524, 762]], device='cuda:0'), 'labels': tensor([20], device='cuda:0')}]
[{'boxes': tensor([[ 221.5170,  496.7585,  858.3901, 1288.0369]], device='cuda:0'), 'labels': tensor([14], device='cuda:0'), 'scores': tensor([0.9302], device='cuda:0')}] [{'boxes': tensor([[ 274,  532,  849, 1287]], device='cuda:0'), 'labels': tensor([14], device='cuda:0')}]
[{'boxes': tensor([[ 779.8517,  325.9402, 1276.5173,  781.1140]], device='cuda:0'), 'labels': tensor([14], device='cuda:0'), 'scores': tensor([0.8573], device='cuda:0')}] [{'boxes': ten

[{'boxes': tensor([[ 96.1589, 195.7538, 617.9790, 626.7090]], device='cuda:0'), 'labels': tensor([0], device='cuda:0'), 'scores': tensor([0.3345], device='cuda:0')}] [{'boxes': tensor([[ 67, 164, 628, 636]], device='cuda:0'), 'labels': tensor([6], device='cuda:0')}]
[{'boxes': tensor([[ 804.1262,  427.7535, 1060.8738, 1013.6845]], device='cuda:0'), 'labels': tensor([0], device='cuda:0'), 'scores': tensor([0.6932], device='cuda:0')}] [{'boxes': tensor([[ 857,  413, 1075, 1024]], device='cuda:0'), 'labels': tensor([6], device='cuda:0')}]
[{'boxes': tensor([[  0.9899,   3.8611, 432.0000, 323.5630]], device='cuda:0'), 'labels': tensor([16], device='cuda:0'), 'scores': tensor([0.9004], device='cuda:0')}] [{'boxes': tensor([[ 22,   2, 432, 319]], device='cuda:0'), 'labels': tensor([16], device='cuda:0')}]
[{'boxes': tensor([[  4.5806,  75.8140, 458.9105, 275.2332]], device='cuda:0'), 'labels': tensor([4], device='cuda:0'), 'scores': tensor([0.6480], device='cuda:0')}] [{'boxes': tensor([[ 17

[{'boxes': tensor([[277.6340, 147.7470, 877.6418, 493.2191]], device='cuda:0'), 'labels': tensor([25], device='cuda:0'), 'scores': tensor([0.5380], device='cuda:0')}] [{'boxes': tensor([[280, 173, 863, 497]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[ 83.6381,   1.7616, 379.2545, 283.3681]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.6135], device='cuda:0')}] [{'boxes': tensor([[ 80,   4, 356, 282]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[ 91.6549, 352.6384, 348.6760, 604.3057]], device='cuda:0'), 'labels': tensor([24], device='cuda:0'), 'scores': tensor([0.4026], device='cuda:0')}] [{'boxes': tensor([[ 81, 364, 349, 605]], device='cuda:0'), 'labels': tensor([23], device='cuda:0')}]
[{'boxes': tensor([[  0.0000,  32.1856,  99.3645, 286.1018]], device='cuda:0'), 'labels': tensor([18], device='cuda:0'), 'scores': tensor([0.8078], device='cuda:0')}] [{'boxes': tensor([[  2,  

[{'boxes': tensor([[214.3790, 612.5482, 530.9456, 975.8473]], device='cuda:0'), 'labels': tensor([22], device='cuda:0'), 'scores': tensor([0.8003], device='cuda:0')}] [{'boxes': tensor([[223, 615, 516, 953]], device='cuda:0'), 'labels': tensor([13], device='cuda:0')}]
[{'boxes': tensor([[ 93.9240, 396.2241, 570.7920, 675.5679]], device='cuda:0'), 'labels': tensor([13], device='cuda:0'), 'scores': tensor([0.5151], device='cuda:0')}] [{'boxes': tensor([[103, 392, 567, 656]], device='cuda:0'), 'labels': tensor([13], device='cuda:0')}]
[{'boxes': tensor([[514.1100, 539.5317, 761.9396, 950.1334]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'scores': tensor([0.2263], device='cuda:0')}] [{'boxes': tensor([[223, 615, 516, 953]], device='cuda:0'), 'labels': tensor([13], device='cuda:0')}]
[{'boxes': tensor([[  9.4249, 306.5381, 444.6807, 563.0615]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'scores': tensor([0.2616], device='cuda:0')}] [{'boxes': tensor([[103, 392